In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
Gemini_key = os.getenv("Gemini_key")
Typhoon_key = os.getenv("Typhoon_key")


In [2]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.documents import Document
import re
file_paths = [
    ("Data/ปฏิทินการศึกษา2568.pdf", "calendar"),
    ("Data/รายละเอียดของหลักสูตร.pdf", "curriculum"),
]
def clean_text(text):
    text = text.replace('\uf70b', '้') 
    text = text.replace('\uf70a', '่')
    text = text.replace('\uf701', '')
    text = text.replace('\uf712', '็')
    text = text.replace('\uf70e', '์')  
    text = text.replace('\uf710', 'ั')   
    text = text.replace('\uf702', 'ำ')  
    text = re.sub(r'\s+', ' ', text) 
    return text.strip()
docs = []
for path, source_tag in file_paths:
    loader = PyMuPDFLoader(path)
    loaded_pages = loader.load()
    for i, page in enumerate(loaded_pages):
        raw_text = page.page_content
        cleaned = clean_text(raw_text)
        doc = Document(
            page_content=cleaned,
            metadata={
                "page": i + 1,
                "source": source_tag,
                "file": os.path.basename(path),
            }
        )
        docs.append(doc)

In [3]:
print(f"Loaded {len(docs)} documents from {len(file_paths)} files.")

Loaded 164 documents from 2 files.


In [4]:
docs[105]

Document(metadata={'page': 104, 'source': 'curriculum', 'file': 'รายละเอียดของหลักสูตร.pdf'}, page_content='104 2. การประเมินทักษะของอาจารย์ในการใช้แผนกลยุทธ์การสอน - ประเมินจากนักศึกษาเกี่ยวกับการสอนของอาจารย์ในทุกด้าน เช่น กลวิธีการสอน การตรง ต่อเวลา การชี้แจงเป\uf706าหมาย วัตถุประสงค์ของรายวิชา เกณฑ์การวัดและประเมินผล และ การใช้สื่อการสอน - ประเมินโดยตัวอาจารย์เองและเพื่อนร่วมงาน 6.4.3.2 การประเมินหลักสูตรในภาพรวม มีกระบวนการที่ได้ข้อมูลย้อนกลับในการประเมินคุณภาพของหลักสูตรในภาพรวม เช่น 1. ประเมินหลักสูตรในภาพรวมโดยการทำแบบสอบถามนักศึกษาชั้นปำสุดท้ายและบัณฑิตใหม่ เกี่ยวกับสถานภาพการประกอบอาชีพ 2. ประเมินโดยที่ปรึกษาหรือผู้ทรงคุณวุฒิจากรายงานผลการดำเนินการหลักสูตรประจำปำ 3. ประเมินโดยผู้ใช้บัณฑิตหรือผู้มีส่วนเกี่ยวข้องอื่นๆ 4. สำรวจจำนวนนักศึกษาที่สำเร็จการศึกษา สำรวจข้อมูลเกี่ยวกับการทำงานและการศึกษาต่อ ของบัณฑิต 6.4.3.3 การประเมินผลการดำเนินงานตามรายละเอียดหลักสูตร เป็นไปตามการประเมินคุณภาพหลักสูตรตามหลักเกณฑ์ของ สกอ. 6.4.3.4 การทบทวนผลการประเมินและวางแผนปรับปรุง 1. มีการนำข้อมูลจา

In [5]:
# แบ่งเอกสารเป็นชิ้น
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""],
    is_separator_regex=False,
)
splits = text_splitter.split_documents(docs)

In [ ]:
#from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

vectordb = FAISS.from_documents(
    documents=splits,
    embedding=embedding
)

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    google_api_key=Gemini_key
)

In [8]:
from langchain.tools import tool

@tool
def summarize_subject_structure(texts: list[str]) -> str:
    """จัดหมวดหมู่วิชาออกเป็น: วิชาพื้นฐาน วิชาเฉพาะ วิชาเลือกเสรี กลุ่มวิชาโท และสรุปหน่วยกิตของแต่ละหมวด"""
    joined = "\n".join(texts)
    prompt = f"""
    ข้อมูลต่อไปนี้มีรายละเอียดของวิชาในหลักสูตรวิทยาการคอมพิวเตอร์:
    ----
    {joined}
    ----
    กรุณาสรุปข้อมูลดังนี้:
    1. หมวดวิชา: เช่น วิชาพื้นฐาน, วิชาเฉพาะ, วิชาเลือกเสรี, กลุ่มวิชาโท
    2. ชื่อวิชาแต่ละหมวด
    3. หน่วยกิตรวมของแต่ละหมวด
    4. รวมหน่วยกิตทั้งหมด
    """
    print("Summarizing subject structure...")
    result = llm.invoke(prompt)
    return result.content.strip()

In [9]:

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import Tool
multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(search_kwargs={"k": 12}),
    llm=llm
)

retriever_tool = create_retriever_tool(
    retriever=multi_query_retriever, # ใช้ retriever ตัวใหม่
    name="university_document_search",
    description="ใช้สำหรับค้นหาข้อมูลจากเอกสารเกี่ยวกับรายละเอียดของหลักสูตรวิทยาการคอมพิวเตอร์และปฏิทินการศึกษา"
)

summarize_subject_structure_tool = Tool(
    name="summarize_subject_structure",
    description="ใช้เพื่อสรุปหมวดหมู่วิชา โครงสร้างหลักสูตร รายวิชา และจำนวนหน่วยกิต",
    func=summarize_subject_structure
)

In [10]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

# สร้าง Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "ผู้ใช้ทุกคนถามเกี่ยวกับหลักสูตรหรือเอกสารที่คุณมี\n"
     "คุณต้องตอบคำถามโดยใช้ข้อมูลจากเครื่องมือ 'university_document_search' เท่านั้น ห้ามใช้ความรู้ภายนอกเด็ดขาด\n"
     "วิเคราะห์คำถาม: ทำความเข้าใจว่าผู้ใช้ต้องการทราบข้อมูลอะไร เช่น รหัสวิชา, หน่วยกิต, วันที่, หรือข้อมูลภาพรวม\n"
     "สังเคราะห์คำตอบ: อ่านข้อมูลที่เครื่องมือค้นหามาให้ทั้งหมดอย่างละเอียด แม้ข้อมูลจะดูไม่เป็นระเบียบหรือมาจากตารางที่แตกไปบ้าง ให้พยายามทำความเข้าใจและสกัดเฉพาะส่วนที่ตอบคำถามของผู้ใช้\n"
     "หากคำถามของผู้ใช้เกี่ยวกับหมวดหมู่วิชา, รายชื่อวิชา, หน่วยกิต หรือโครงสร้างหลักสูตรโดยรวม ให้ใช้ summarize_subject_structure เพื่อจัดหมวด"
     "ตอบอย่างมั่นใจ: ตอบเป็นภาษาไทยที่กระชับ ครบถ้วนและชัดเจน หากไม่พบข้อมูลจริงๆ ให้ตอบว่า 'ไม่พบข้อมูลที่เกี่ยวข้องในเอกสาร'\n"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])
# สร้าง Agent
tools = [summarize_subject_structure_tool,retriever_tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [ ]:
def verify_answer(llm, question: str, answer: str, context: str) -> bool:
    verify_prompt = f"""
    คุณเป็นผู้ตรวจสอบความถูกต้องของคำตอบจากเอกสารมหาวิทยาลัย
    คำถาม: {question}
    คำตอบ: {answer}
    ข้อมูลอ้างอิงจากเอกสาร: {context}
    กรุณาเช็คอย่างละเอียด คำตอบข้างต้น มีข้อมูลสนับสนุนอยู่ในข้อความอ้างอิงหรือไม่:
    - ถ้ามีข้อมูลสนับสนุนให้ตอบว่า "YES"
    - ถ้าบางส่วนของคำตอบไม่มีข้อมูลสนับสนุนในข้อความอ้างอิง หรือคำตอบ **ไม่ถูกต้องตามข้อมูลอ้างอิง** ให้ตอบว่า "NO"
    - หากคำตอบเป็นการระบุว่า 'ไม่พบข้อมูล' ให้พิจารณาว่า ถูกต้อง ถ้าบริบทไม่มีข้อมูลนั้นจริงๆ ให้ตอบว่า "YES"
    ตอบเฉพาะ "YES" หรือ "NO" เท่านั้น
    """
    try:
        response = llm.invoke(verify_prompt)
        result = response.content.strip().upper()
        return result.startswith("YES")
    except Exception as e:
        print("ตรวจสอบคำตอบล้มเหลว:", e)
        return False



In [12]:
question = "มีวิชาโทอะไรบ้าง"
response = agent_executor.invoke({"input": question})
raw_answer = response["output"]

retrieved_docs = vectordb.similarity_search(question, k=12)
if not retrieved_docs:
    print("ไม่พบข้อมูลที่เกี่ยวข้องในเอกสาร")
else:
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    #print(f"คำตอบจาก Agent: {raw_answer}")
    #print(f"Context ที่ใช้ตรวจสอบ: \n{context}...") 
    is_valid = verify_answer(llm, question, raw_answer, context)
    if is_valid:
        print(raw_answer)
    else:
        print("ไม่สามารถตอบได้")



> Entering new AgentExecutor chain...

Invoking: `university_document_search` with `{'query': 'วิชาโท'}`


137 หลักสูตร ฉบับ พ.ศ. 2561 หลักสูตร ฉบับ พ.ศ. 2566 สรุปการ เปลี่ยนแปลง 4. วิชาโท 4.1. วิชาโทวิทยาการคอมพิวเตอร์ (สำหรับนักศึกษานอกสาขาวิชาวิทยาการ คอมพิวเตอร์เท่านั้น) นักศึกษาผู้ประสงค์จะศึกษาวิชาวิทยาการ คอมพิวเตอร์เป็นวิชาโท ต้องศึกษารายวิชาใน สาขาวิชาวิทยาการคอมพิวเตอร์ ไม่น้อยกว่า 15 หน่วยกิต ตามเงื่อนไขดังต่อไปนี้ 1. นักศึกษาต้องศึกษา 4 วิชา รวม 12 หน่วย กิต จากรายวิชาต่อไปนี้ คพ.103 การโปรแกรมคอมพิวเตอร์เบื้องต้น คพ.112 การโปรแกรมเชิงวัตถุเบื้องต้น คพ.213 โครงสร้างข้อมูล คพ.251 ระบบฐานข้อมูล 1 และต้องสอบได้ไม่ต่ำกว่าระดับ C ในวิชา คพ.103 และ คพ.112 2. นักศึกษาต้องศึกษารายวิชาต่างๆ ใน สาขาวิชาวิทยาการคอมพิวเตอร์ ระดับ คพ.2xx หรือ คพ.3xx หรือ คพ.4xx อีกไม่น้อยกว่า 6 หน่วยกิต ทั้งนี้ ไม่นับรายวิชา คพ.285 คพ.300 และ คพ.301 โดยจะต้องสอบไล่ได้ค่าระดับเฉลี่ย ไม่ต่ำกว่า 2.00 ของกลุ่มรายวิชาโทในสาขาวิชา วิทยาการคอมพิวเตอร์ 4. วิชาโท 4.1. วิชาโทวิทยาการคอมพิวเตอร์ (สำหรับนักศึกษาน